In [10]:
import scanpy as sc
import scvi
import numpy as np
import anndata as ad
from anndata import AnnData
from scipy import sparse
import pandas as pd

In [27]:
data_dir = "/omics/groups/OE0533/internal/katharina/scDoRI/gastrulation_data_new/Kathi/"
rna_adata =  sc.read_h5ad(data_dir + "rna_anndata_from_seurat")
atac_adata = sc.read_h5ad(data_dir + "peak_anndata_from_archr")

In [37]:
#rna_adata =  sc.read_h5ad(data_dir + "rna_anndata_from_seurat")

#atac_adata = sc.read_h5ad(data_dir + "peak_anndata_from_archr")

In [28]:
# make sure that the cells are in the same order
atac_adata = atac_adata[rna_adata.obs.index.tolist(), :]

In [29]:
atac_df = atac_adata.var
#atac_df["modality"]=["Peaks" for i in range(len(atac_df))]

In [31]:
atac_df.insert(1, "modality", ["Peaks" for i in range(len(atac_df))])

In [40]:
rna_df = rna_adata.var
#rna_df = rna_df.rename(columns = {"gene":"ID"})
rna_df.insert(0, "modality", ["Gene_expression" for i in range(len(rna_df))])
#rna_df["modality"] = ["Gene_expression" for i in range(len(rna_df))]

In [42]:
df = rna_df.append(atac_df)

In [44]:
np.all(rna_adata.obs.index == atac_adata.obs.index)

True

In [45]:
matrix = sparse.hstack((rna_adata.X, atac_adata.X))

In [46]:
rna_adata.uns

OverloadedDict, wrapping:
	OrderedDict()
With overloaded keys:
	['neighbors'].

In [47]:
multiome = AnnData(
X = matrix,
obs = atac_adata.obs,
var = df)

In [48]:
multiome

AnnData object with n_obs × n_vars = 62963 × 224536
    obs: 'BlacklistRatio', 'nDiFrags', 'nFrags', 'nMonoFrags', 'nMultiFrags', 'NucleosomeRatio', 'PassQC', 'PromoterRatio', 'ReadsInBlacklist', 'ReadsInPromoter', 'ReadsInTSS', 'Sample', 'TSSEnrichment', 'barcode', 'sample', 'nFeature_RNA', 'nCount_RNA', 'mitochondrial_percent_RNA', 'ribosomal_percent_RNA', 'stage', 'genotype', 'pass_rnaQC', 'doublet_score', 'doublet_call', 'celltype.mapped', 'celltype.score', 'closest.cell', 'TSSEnrichment_atac', 'ReadsInTSS_atac', 'PromoterRatio_atac', 'NucleosomeRatio_atac', 'nFrags_atac', 'BlacklistRatio_atac', 'ReadsInPeaks', 'FRIP'
    var: 'modality', 'peak', 'seqnames', 'start', 'end', 'width', 'strand', 'score', 'replicateScoreQuantile', 'groupScoreQuantile', 'Reproducibility', 'GroupReplicate', 'distToGeneStart', 'nearestGene', 'peakType', 'distToTSS', 'nearestTSS', 'GC', 'idx', 'N'

In [49]:
adata_mvi = scvi.data.organize_multiome_anndatas(multiome)


In [50]:
adata_mvi.var

,modality,peak,seqnames,start,end,width,strand,score,replicateScoreQuantile,groupScoreQuantile,Reproducibility,GroupReplicate,distToGeneStart,nearestGene,peakType,distToTSS,nearestTSS,GC,idx,N
Xkr4,Gene_expression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gm1992,Gene_expression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gm19938,Gene_expression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gm37381,Gene_expression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rp1,Gene_expression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192247,Peaks,chrX:169902806-169903406,chrX,169902806.0,169903406.0,601.0,*,9.54947,0.251,0.083,2.0,Spinal_cord._.Rep2,123470.0,4933400A11Rik,Intronic,23486.0,uc009rco.1,0.3627,3284.0,0.0
192248,Peaks,chrX:169905921-169906521,chrX,169905921.0,169906521.0,601.0,*,60.40740,0.857,0.790,2.0,Forebrain_Midbrain_Hindbrain._.Rep1,126585.0,4933400A11Rik,Intronic,20732.0,uc009rco.1,0.4642,3285.0,0.0
192249,Peaks,chrX:169915616-169916216,chrX,169915616.0,169916216.0,601.0,*,11.75670,0.304,0.088,2.0,Endothelium._.Rep2,136280.0,4933400A11Rik,Intronic,11037.0,uc009rco.1,0.5008,3286.0,0.0
192250,Peaks,chrX:169925487-169926087,chrX,169925487.0,169926087.0,601.0,*,18.78800,0.557,0.358,2.0,Spinal_cord._.Rep1,146151.0,4933400A11Rik,Intronic,1166.0,uc009rco.1,0.5058,3287.0,0.0


In [51]:
adata_mvi = adata_mvi[:, adata_mvi.var["modality"].argsort()].copy()


In [52]:
scvi.model.MULTIVI.setup_anndata(adata_mvi, batch_key='modality')

/omics/groups/OE0533/internal/katharina/scvi2/lib/python3.7/site-packages/scvi/data/_utils.py:102: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  "Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization."


In [53]:
mvi = scvi.model.MULTIVI(
    adata_mvi,
    n_genes=(adata_mvi.var['modality']=='Gene_expression').sum(),
    n_regions=(adata_mvi.var['modality']=='Peaks').sum(),
)
mvi.view_anndata_setup()

Anndata setup with scvi-tools version 0.15.0.

Setup via `MULTIVI.setup_anndata` with arguments:

{
│   'layer': None,
│   'batch_key': 'modality',
│   'labels_key': None,
│   'size_factor_key': None,
│   'categorical_covariate_keys': None,
│   'continuous_covariate_keys': None
}

         Summary Statistics          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃     Summary Stat Key     ┃ Value  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│         n_cells          │ 62963  │
│          n_vars          │ 224536 │
│         n_batch          │   1    │
│         n_labels         │   1    │
│ n_extra_categorical_covs │   0    │
│ n_extra_continuous_covs  │   0    │
└──────────────────────────┴────────┘

               Data Registry                
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Registry Key ┃    scvi-tools Location    ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      X       │          adata.X          │
│    batch     │ adata.obs['_scvi_batch']  │
│    labels    │ adata.obs['_scvi_labels'] │
└──────────────┴───────────────────────────┘

                    batch State Registry                    
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃    Source Location    ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['modality'] │   paired   │          0          │
└───────────────────────┴────────────┴─────────────────────┘

                     labels State Registry                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃      Source Location      ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['_scvi_labels'] │     0      │          0          │
└───────────────────────────┴────────────┴─────────────────────┘

In [54]:
mvi.train()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Epoch 1/500:   0%|          | 0/500 [00:00<?, ?it/s]

/omics/groups/OE0533/internal/katharina/scvi2/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [4]:
rna_adata.obs

,orig.ident,nCount_RNA,nFeature_RNA,barcode,sample,mitochondrial_percent_RNA,ribosomal_percent_RNA,stage,genotype,pass_rnaQC,...,ReadsInTSS_atac,PromoterRatio_atac,NucleosomeRatio_atac,nFrags_atac,BlacklistRatio_atac,ReadsInPeaks,FRIP,RNA_snn_res.0.6,seurat_clusters,ident
E7.5_rep1#CAAGTTATCGCACAAT-1,E7.5,18561,4717,CAAGTTATCGCACAAT-1,E7.5_rep1,31.33,9.93,E7.5,WT,True,...,10179.0,0.10,2.80,243740.0,0.02,100015.0,0.205277,9,9,9
E7.5_rep1#TTTAACGAGTTAGACC-1,E7.5,4212,2345,TTTAACGAGTTAGACC-1,E7.5_rep1,3.70,10.85,E7.5,WT,True,...,11419.0,0.14,1.10,178160.0,0.02,96404.0,0.270795,8,8,8
E7.5_rep1#GTATTGCAGCGGATTT-1,E7.5,6259,3394,GTATTGCAGCGGATTT-1,E7.5_rep1,2.05,7.56,E7.5,WT,True,...,11298.0,0.14,1.31,174003.0,0.02,92711.0,0.266497,9,9,9
E7.5_rep1#GCTGTACCACTATGGC-1,E7.5,16266,5459,GCTGTACCACTATGGC-1,E7.5_rep1,4.62,5.26,E7.5,WT,True,...,15781.0,0.18,1.25,169021.0,0.02,121526.0,0.359625,3,3,3
E7.5_rep1#ATTAGCTCACCCACAG-1,E7.5,22993,6041,ATTAGCTCACCCACAG-1,E7.5_rep1,15.31,6.05,E7.5,WT,True,...,10460.0,0.14,1.08,162094.0,0.02,86649.0,0.267386,7,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E8.5_CRISPR_T_WT#TTCACTGTCAAGGACA-1,E8.5,12792,2359,TTCACTGTCAAGGACA-1,E8.5_CRISPR_T_WT,23.92,11.63,E8.5,WT,True,...,148.0,0.25,0.92,1009.0,0.01,877.0,0.435020,5,5,5
E8.5_CRISPR_T_WT#CAGTATGGTTCGCTCA-1,E8.5,24500,4347,CAGTATGGTTCGCTCA-1,E8.5_CRISPR_T_WT,8.71,11.53,E8.5,WT,True,...,122.0,0.26,0.72,1008.0,0.01,938.0,0.465278,14,14,14
E8.5_CRISPR_T_WT#GTTAAGTGTTGCGTAA-1,E8.5,32630,5069,GTTAAGTGTTGCGTAA-1,E8.5_CRISPR_T_WT,7.91,12.53,E8.5,WT,True,...,177.0,0.28,0.71,1005.0,0.02,951.0,0.473134,13,13,13
E8.5_CRISPR_T_WT#ATCGCTTGTTCACCAT-1,E8.5,38912,5533,ATCGCTTGTTCACCAT-1,E8.5_CRISPR_T_WT,5.94,9.62,E8.5,WT,True,...,119.0,0.19,1.05,1002.0,0.01,878.0,0.438124,12,12,12


In [5]:
#adata.var_names_make_unique()
rna_adata.var_names_make_unique()
atac_adata.var_names_make_unique()

In [6]:
old = scvi.data.read_h5ad("/omics/groups/OE0533/internal/katharina/scDoRI/gastrulation_data/old_data_three_timepoints/anndata_rna.h5ad")

In [7]:
rna_adata = rna_adata[:,rna_adata.var.index.isin(old.var.index)]

In [8]:
annotations = pd.merge(old.var, rna_adata.var, on="gene", how = "inner")
annotations.index = old.var.index

In [9]:
annotations = annotations.drop(["gene", "Strand"], axis=1)

In [10]:
annotations = annotations.rename(columns={"Accession":"ID", "End":"end", "Start":"start", "Chromosome":"chr"})

In [11]:
rna_adata.var = annotations

In [12]:
matrix = sparse.hstack((rna_adata.X, atac_adata.X))

In [13]:
rna_df = rna_adata.var
#rna_df = rna_df.rename(columns = {"gene":"ID"})
rna_df["modality"] = ["Gene_expression" for i in range(len(rna_df))]
rna_df

,ID,chr,end,start,modality
Xkr4,ENSMUSG00000051951,1,3671498,3205901,Gene_expression
Gm1992,ENSMUSG00000089699,1,3513553,3466587,Gene_expression
Gm19938,ENSMUSG00000102331,1,3658904,3647309,Gene_expression
Gm37381,ENSMUSG00000102343,1,3986215,3905739,Gene_expression
Rp1,ENSMUSG00000025900,1,4409241,3999557,Gene_expression
...,...,...,...,...,...
AC124606.1,ENSMUSG00000095523,JH584299.1,840451,837364,Gene_expression
AC133095.2,ENSMUSG00000095475,JH584299.1,913083,910289,Gene_expression
AC133095.1,ENSMUSG00000094855,JH584299.1,924675,921942,Gene_expression
AC234645.1,ENSMUSG00000095019,JH584303.1,82689,81607,Gene_expression


In [14]:
atac_df = atac_adata.var
atac_df = atac_df.rename(columns={"idx":"ID"})
atac_df["modality"]=["Peaks" for i in range(len(atac_df))]

In [15]:
atac_df = atac_df.drop(["score"], axis = 1)

In [16]:
atac_df

,chr,start,end,ID,modality
chr1:3035600-3036200,chr1,3035600,3036200,chr1:3035600-3036200,Peaks
chr1:3062691-3063291,chr1,3062691,3063291,chr1:3062691-3063291,Peaks
chr1:3072272-3072872,chr1,3072272,3072872,chr1:3072272-3072872,Peaks
chr1:3191513-3192113,chr1,3191513,3192113,chr1:3191513-3192113,Peaks
chr1:3466250-3466850,chr1,3466250,3466850,chr1:3466250-3466850,Peaks
...,...,...,...,...,...
chrX:169883288-169883888,chrX,169883288,169883888,chrX:169883288-169883888,Peaks
chrX:169899093-169899693,chrX,169899093,169899693,chrX:169899093-169899693,Peaks
chrX:169906016-169906616,chrX,169906016,169906616,chrX:169906016-169906616,Peaks
chrX:169915632-169916232,chrX,169915632,169916232,chrX:169915632-169916232,Peaks


In [17]:
df = rna_df.append(atac_df)

In [18]:
multiome = AnnData(
X = matrix,
obs = atac_adata.obs,
var = df,
uns = rna_adata.uns)

In [19]:
multiome.shape

(45991, 212744)

In [20]:
adata_mvi = scvi.data.organize_multiome_anndatas(multiome)


In [21]:
adata_mvi = adata_mvi[:, adata_mvi.var["modality"].argsort()].copy()


In [22]:
print(adata_mvi.shape)
sc.pp.filter_genes(adata_mvi, min_cells=int(adata_mvi.shape[0] * 0.01))
print(adata_mvi.shape)

(45991, 212744)
(45991, 186627)


In [23]:
scvi.model.MULTIVI.setup_anndata(adata_mvi, batch_key='modality')

/omics/groups/OE0533/internal/katharina/scvi2/lib/python3.7/site-packages/scvi/data/_utils.py:102: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  "Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization."


In [26]:
mvi = scvi.model.MULTIVI(
    adata_mvi,
    n_genes=(adata_mvi.var['modality']=='Gene_expression').sum(),
    n_regions=(adata_mvi.var['modality']=='Peaks').sum(),
)
mvi.view_anndata_setup()

Anndata setup with scvi-tools version 0.15.0.

Setup via `MULTIVI.setup_anndata` with arguments:

{
│   'layer': None,
│   'batch_key': 'modality',
│   'labels_key': None,
│   'size_factor_key': None,
│   'categorical_covariate_keys': None,
│   'continuous_covariate_keys': None
}

         Summary Statistics          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃     Summary Stat Key     ┃ Value  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│         n_cells          │ 45991  │
│          n_vars          │ 186627 │
│         n_batch          │   1    │
│         n_labels         │   1    │
│ n_extra_categorical_covs │   0    │
│ n_extra_continuous_covs  │   0    │
└──────────────────────────┴────────┘

               Data Registry                
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Registry Key ┃    scvi-tools Location    ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      X       │          adata.X          │
│    batch     │ adata.obs['_scvi_batch']  │
│    labels    │ adata.obs['_scvi_labels'] │
└──────────────┴───────────────────────────┘

                    batch State Registry                    
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃    Source Location    ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['modality'] │   paired   │          0          │
└───────────────────────┴────────────┴─────────────────────┘

                     labels State Registry                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃      Source Location      ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['_scvi_labels'] │     0      │          0          │
└───────────────────────────┴────────────┴─────────────────────┘

In [ ]:
mvi.train()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Epoch 1/500:   0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
#multiome_mvi = scvi.data.organize_multiome_anndatas(multiome)

In [ ]:
#multiome_mvi.var

In [ ]:
#adata_mvi = scvi.data.organize_multiome_anndatas(multiome)

In [ ]:
#multiome.write("multiome_gastr.h5ad", compression="gzip")

In [ ]:
#multiome_mvi = multiome_mvi[:, multiome_mvi.var["modality"].argsort()].copy()

In [ ]:
#multiome_mvi.var

Filter features which appear in fewer than 1% of cells.

The main batch key should correspond to the modality of the cells. All other batch annotations are added via categorical_covariate_keys.

In [ ]:
test = scvi.data.organize_multiome_anndatas(multiome)

In [ ]:
scvi.model.MULTIVI.setup_anndata(adata)#,
                                # batch_key = "modality")
                                 #categorical_covariate_keys=["sample"])

We need to specify how many features are genes and how many are genomic regions. 

In [ ]:
mvi = scvi.model.MULTIVI(
    adata,
    n_genes=(adata.var['modality']=='Gene_expression').sum(),
    n_regions=(adata.var['modality']=='Peaks').sum()
    #dropout_rate=0.2,
    #n_layers_encoder=2,
    #n_layers_decoder=2,
    #n_latent=20
)

arches_params = dict(
    n_genes=(multiome.var["modality"]=="Gene_expression").sum(),
    n_regions=(multiome.var["modality"]=="Peaks").sum(),
    #dropout_rate=0.2,
    #n_layers_encoder=2,
    #n_layers_decoder=2,
    #n_latent=20,
    #latent_distribution='normal'
)

mvi = scvi.model.MULTIVI(
    multiome,
    **arches_params
)
mvi.view_anndata_setup()

In [ ]:
mvi.train()